In [58]:
%run _jupyter_sounds.py  # Use play_sound_success() at the end of a long cell
%run _jupyter_display.py
%load_ext autoreload
%autoreload 2

In [82]:
####
# Made by Ruben 2024-11-06 
####
from typing import Any, Dict, List, Tuple, Union, Literal, Optional, Type, TypeVar, Generic, Callable, Iterable, Mapping, Sequence, Set, Deque, NamedTuple, ForwardRef, Annotated
from pydantic import BaseModel, Field, ValidationError, validator, TypeAdapter

# import BeforeValidator 
from pydantic import BaseModel, Field, ValidationError, validator, BeforeValidator, AfterValidator

# import Annotated
from typing import Annotated

from pydantic import TypeAdapter, validate_call

from annotated_types import Gt, Ge, Lt, Le, MinLen, MaxLen, Len #,  #, NegativeFloat, NegativeInt, PositiveFloat, PositiveInt, NonNegativeFloat, NonNegativeInt, NonPositiveFloat, NonPositiveInt, EmailStr, NameEmail, UrlStr, UUID1, UUID3, UUID4, UUID5, UUID6, UUID, IPvAnyAddress

from fastcore.test import test_eq  , test_fail
# ------------------------------
# Annotated:

# Allows for 
# simple|complex type definitions, 
# with pre & post validation operations
# and data transformation|coercion
# ------------------------------
# TypeAdapter

# TypeAdapter allows to validate an input with a given type.
# and to transform the input to the desired type.
# ------------------------------
# Using both we can define and enforce typing rules

In [99]:
# Importing Annotated
# depending on your Python version you do one of the following:

from typing import Annotated # Latest Python 3.9
# or
from typing_extensions import Annotated # Python 3.7, 3.8

In [77]:
# Annotated:
# Allows for
# simple|complex type definitions,

Name=Annotated[str, AfterValidator(lambda x: x.title())]

Age=Annotated[int,Ge(0)]

Score0to10=Annotated[int, Gt(0), Le(10)]

# with pre & post validation operations
# and data transformation| coercion

def strip_and_lower(s: str) -> str:
    return s.strip().lower()


def strip_and_upper(s: str) -> str:
    return s.strip().upper()


BeforeStripLower = BeforeValidator(strip_and_lower)
BeforeStripUpper = BeforeValidator(strip_and_upper)

LanguageCode=Annotated[Literal["en","es","it","fr"], BeforeStripLower]

# However you may not need Annotated

Formality=Literal['formal', 'informal',None]

Orientation= Literal["S","SW","SE","N","NE","NW","W","E"]

In [108]:
# Validation on the fly with TypeAdapter is almost as easy as 
# assigning a value to a variable.

my_int="3.0"
my_int=TypeAdapter(int).validate_python(my_int)
my_int

# This fails
try:
    my_int="3.14"
    my_int=TypeAdapter(int).validate_python(my_int)
except ValueError as e: 
    print(e)    


my_float="3.14"
my_float=TypeAdapter(float).validate_python(my_float)
my_float

my_str="33.88"
my_str=TypeAdapter(str).validate_python(my_str)
my_str

score=3
score=TypeAdapter(Score0to10).validate_python(score)
score

# This fails
try:
    score=13
    score=TypeAdapter(Score0to10).validate_python(score)
except ValueError as e:
    print(e)


x=3
age = TypeAdapter(Age).validate_python(x)
age

# Even this works (str with spaces to int)
x=" 88 "
age= TypeAdapter(Age).validate_python(x)
age

# Name gets transformed to title case
x="john"
name=TypeAdapter(Name).validate_python(x)
name

x="formal"
formality=TypeAdapter(Formality).validate_python(x)
formality

# you can even change things on the fly (notice List[Something] )
langs=[" en ","ES   ","  It","    Fr"]
langs=TypeAdapter(List[LanguageCode]).validate_python(langs)
langs



3

1 validation error for int
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='3.14', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


3.14

'33.88'

3

...

3

88

'John'

'formal'

['en', 'es', 'it', 'fr']

In [96]:

# The Before / After validators allow you to do some pre or post processing on the data

# This fails
try:
    TypeAdapter(Formality).validate_python("  formal  ")
except Exception as e:
    print(e)

#ValidationError: 1 validation error for literal['formal','informal',None]
#  Input should be 'formal', 'informal' or None 

# This fails
try:
    TypeAdapter(Orientation).validate_python("s")
except Exception as e:
    print(e)


# But not this
CleanOrientation=Annotated[Orientation, BeforeStripUpper]

orientation="s"
orientation=TypeAdapter(CleanOrientation).validate_python(orientation)
orientation

# But not this
lang=" eS "
lang=TypeAdapter(LanguageCode).validate_python(lang)
lang

1 validation error for literal['formal','informal',None]
  Input should be 'formal', 'informal' or None [type=literal_error, input_value='  formal  ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/literal_error
1 validation error for literal['S','SW','SE','N','NE','NW','W','E']
  Input should be 'S', 'SW', 'SE', 'N', 'NE', 'NW', 'W' or 'E' [type=literal_error, input_value='s', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/literal_error


'S'

'es'

In [97]:
# The validation can be incorporated into your functions just by including
# @validate_call

@validate_call
def greet(name: Name, 
age:     Age, formality: Formality, 
lang: LanguageCode, 
orientation: CleanOrientation):
    print(f"Hello {name} ({age})")
    print(f"Formality: {formality}")
    print(f"Language: {lang}")
    print(f"Orientation: {orientation}")

greet(name="  john  ",age= 33,formality ="formal",lang= "es",orientation= "s")

Hello   John   (33)
Formality: formal
Language: es
Orientation: S


In [98]:
# You don't need complex/strict typing rules to use @validate_call

@validate_call
def greet(name: str, age: int, formality: str, lang: str, orientation: str):
    print(f"Hello {name} ({age})")
    print(f"Formality: {formality}")
    print(f"Language: {lang}")
    print(f"Orientation: {orientation}")

# This will work
greet(name="  john  ",age= 33,formality ="formal",lang= "es",orientation= "s")


# This will fail
try:
    greet(name=["john  "], age=33.3, formality="not formal", lang="ko", orientation=3)
#try:
except Exception as e:
    print(e)

Hello   john   (33)
Formality: formal
Language: es
Orientation: s
3 validation errors for greet
name
  Input should be a valid string [type=string_type, input_value=['john  '], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
age
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=33.3, input_type=float]
    For further information visit https://errors.pydantic.dev/2.8/v/int_from_float
orientation
  Input should be a valid string [type=string_type, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
